In [2]:
import pandas as pd
import numpy as np
import random as rand
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn import  metrics   #Additional sklearn functions

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rand
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier



def get_percent_full(col_name, df):
    return len(df[df[col_name].isnull()][col_name])/len(df[col_name]) * 100.0

def get_uniques(col_name, df):
    return df[col_name].unique()


ins_df = pd.read_csv("/datasets/extr.csv", index_col=0)



# assegnazione di nan agli zeri privi di significato
ins_df["POTENZA"].replace(to_replace=0, value=np.nan, inplace=True)
ins_df["VALORE"].replace(to_replace=0, value=np.nan, inplace=True)
ins_df["CILINDRATA"].replace(to_replace=0, value=np.nan, inplace=True)

# visualizzazione delle percentuali di valori assenti
#for col_name in ins_df.columns.to_list():
#    print(col_name, " : ", get_percent_full(col_name, ins_df))


# valorizzazione dei campi assenti con la media per una certa feature
ins_df["POTENZA"].fillna(ins_df["POTENZA"].mean(), inplace=True  )
ins_df["CILINDRATA"].fillna(ins_df["CILINDRATA"].mean(), inplace=True  )
ins_df["VALORE"].fillna(ins_df["VALORE"].mean(), inplace=True  )
ins_df["STATOCIVILE"].fillna("NON_SPECIFICATO", inplace=True  )



# elementi della stessa categoria raggruppati sotto un' unica classe, in questo caso le chiavi sono mappate sui nuovi valori
stato_mapping = { "NUBILE" : "CELIBE",
                    "CONIUGATA" : "CONIUGATO",
                    "VEDOVA" : "VEDOVO",
                    "DIVORZIATA" : "DIVORZIATO",
                    "SEPARATA" : "SEPARATO"}

# rimpiazzamento dei campi con data chiave col valore corrispondente
ins_df["STATOCIVILE"].replace(to_replace=stato_mapping, inplace=True)




ins_df["ETACONTRAENTE"] = ins_df["ETACONTRAENTE"].clip(lower=18, upper=94)
#'''
# clipped min age and max age
min_age = 18
max_age = 94

age_as_index_to_interval = np.zeros(max_age + 1,dtype=int)

step = 7
count = 0
group = 0
for age in range(min_age, max_age+1):
    if (count+1) %(step + 1) == 0:
        group += 1
        count = 0
    age_as_index_to_interval[age] = group
    count += 1


#print(age_as_index_to_interval[min_age:])



ins_df["INTERVALLO_ETA"] = ins_df["ETACONTRAENTE"].map(lambda x :age_as_index_to_interval[x] )


ins_df.drop(columns="ETACONTRAENTE", axis=1, inplace=True)
#'''



#for col_name in ins_df.columns.to_list():
#    print(col_name, " : ", get_percent_full(col_name, ins_df))

    
# test delle performance degli algoritmi a seguito dell' eliminazione di una o piu delle seguenti colonne
#ins_df.drop(columns="VALORE", axis=1, inplace=True)
#ins_df.drop(columns="STATOCIVILE", axis=1, inplace=True)
#ins_df.drop(columns="ANTIFURTO", axis=1, inplace=True)
    
continuous_columns = [#"ETACONTRAENTE",
                      "ETAVEICOLO", 
                      "POTENZA",
                      "CILINDRATA",
                      "VALORE"]

categorical_columns = [ "INTERVALLO_ETA",
                        "COMUNE",
                       "PROVINCIA",
                       "SESSO", 
                       "CAP",
                       "ANTIFURTO",
                       "ALIMENTAZIONE",
                       "STATOCIVILE" ]

classes = ["RC", "INC", "FUR", "ESP", "CRI", "EVN", "INF", "ASS", "TUT" ]

# standardizzazione feature a valori continui
std = StandardScaler() 
ins_df[continuous_columns] = std.fit_transform(ins_df[continuous_columns])

# one hot encoding delle colonne categoriche specificate
ins_df = pd.get_dummies(ins_df, columns=categorical_columns )

X = ins_df.drop(columns=classes, axis=1)

to_predict = ["INC", "FUR", "ESP", "CRI", "EVN", "INF", "ASS", "TUT" ]

for col_to_predict in to_predict:
    
    y = ins_df[col_to_predict]


    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
    '''
    model = xgb.XGBClassifier(max_depth=4, n_estimators=2000, learning_rate=0.01, random_state=42)
    '''
    '''
    model = GradientBoostingClassifier(loss="deviance",
                                    learning_rate=0.04, n_estimators=1000,
                                    min_samples_split=2,
                                    min_samples_leaf=1,
                                    max_depth=3,
                                    random_state=42)
    
    '''
    #'''
    model = DecisionTreeClassifier(criterion="entropy",
                                   max_depth=9)
    #'''
    '''
    eoe_dtree = DecisionTreeClassifier(criterion="entropy",
                                   max_depth=6)
    model = AdaBoostClassifier(base_estimator= eoe_dtree,
                        n_estimators=200,
                        learning_rate=0.05,
                        random_state=42)
    '''                    
                        
    '''
    model = RandomForestClassifier(bootstrap=True, criterion="entropy", max_depth=10,
                                max_features="sqrt", min_samples_leaf=1, min_samples_split=10,
                                n_estimators=1000, n_jobs=-1, random_state=42)
    '''
    model.fit(x_train, y_train)
    accuracy = model.score(x_test, y_test)
    print("accuracy : ", accuracy)
   



accuracy :  1.0
accuracy :  0.954
accuracy :  0.9195
accuracy :  0.771
accuracy :  0.9327
accuracy :  0.9279
accuracy :  0.6232
accuracy :  0.646
